# CS 559 Homework 3

Generate data and split into test and train sets

In [1]:
from sklearn import datasets
import numpy as np
X, y = datasets.make_blobs(n_samples = 100, n_features = 4, centers = 2, cluster_std = 1.5, random_state = 123)
y = np.array([-1 if i == 0 else 1 for i in y])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)

Training set size: (80, 4)
Test set size: (20, 4)


# LDA

Write a method that computes the between-class scatter matrix SB . Compute SB using a train data.

In [2]:
def between_class_scatter(X_train, y_train):
    mean_overall = np.mean(X_train, axis=0)
    classes = np.unique(y_train)
    S_B = np.zeros((X_train.shape[1], X_train.shape[1]))

    for c in classes:
        X_c = X_train[y_train == c]
        mean_c = np.mean(X_c, axis=0)
        n_c = X_c.shape[0]
        mean_diff = (mean_c - mean_overall).reshape(-1, 1)
        S_B += n_c * (mean_diff @ mean_diff.T)
    
    return S_B

S_B = between_class_scatter(X_train, y_train)
print("Between-class scatter matrix S_B:\n", S_B)


Between-class scatter matrix S_B:
 [[ 1.42177386e+00 -1.48370842e+01 -7.83733634e+01 -1.38981884e+01]
 [-1.48370842e+01  1.54834094e+02  8.17874222e+02  1.45036139e+02]
 [-7.83733634e+01  8.17874222e+02  4.32022577e+03  7.66118860e+02]
 [-1.38981884e+01  1.45036139e+02  7.66118860e+02  1.35858203e+02]]


Write a method that computes and returns the within-class scatter of each class, the total within-class scatter (Sw), and its inverse (S−1w ). Calculate Sw and S−1w using the train data.

In [3]:
def within_class_scatter(X_train, y_train):
    classes = np.unique(y_train)
    S_W = np.zeros((X_train.shape[1], X_train.shape[1]))

    for c in classes:
        X_c = X_train[y_train == c]
        mean_c = np.mean(X_c, axis=0)
        scatter_c = (X_c - mean_c).T @ (X_c - mean_c)
        S_W += scatter_c
    
    S_W_inverse = np.linalg.inv(S_W)
    return S_W, S_W_inverse

S_W, S_W_inverse = within_class_scatter(X_train, y_train)
print("Within-class scatter matrix S_W:\n", S_W)
print("Inverse of within-class scatter matrix S_W_inv:\n", S_W_inverse)


Within-class scatter matrix S_W:
 [[189.11331817   6.52860091 -16.42044417   8.18216257]
 [  6.52860091 204.02423606  -2.00586952   5.86496479]
 [-16.42044417  -2.00586952 154.61882164   2.54574474]
 [  8.18216257   5.86496479   2.54574474 147.72968011]]
Inverse of within-class scatter matrix S_W_inv:
 [[ 5.35588997e-03 -1.57131974e-04  5.71698050e-04 -3.00255075e-04]
 [-1.57131974e-04  4.91228014e-03  5.01215342e-05 -1.87181528e-04]
 [ 5.71698050e-04  5.01215342e-05  6.53128940e-03 -1.46204087e-04]
 [-3.00255075e-04 -1.87181528e-04 -1.46204087e-04  6.79570108e-03]]


NumPy.linalg.eig(·) estimates eigenvalues, λ and their corresponding eigenvectors u.Using NumPy.linalg.eig(·), find the direction of all projection planes, and transform the train data to each projection plane. Determine the most ideal λ and u for the binary classification.Explain why.

In [4]:
def find_eigenvectors(S_B, S_W_inverse):
    eigvalues, eigvectors = np.linalg.eig(S_W_inverse @ S_B)
    index = np.argsort(eigvalues)[::-1]
    eigvalues = eigvalues[index]
    eigvectors = eigvectors[:, index]
    return eigvalues, eigvectors

eigvalues, eigvectors = find_eigenvectors(S_B, S_W_inverse)
print("Eigenvalues:\n", eigvalues)
print("Eigenvectors:\n", eigvectors)


Eigenvalues:
 [2.96351674e+01+0.00000000e+00j 2.70087633e-16+2.94554556e-16j
 2.70087633e-16-2.94554556e-16j 0.00000000e+00+0.00000000e+00j]
Eigenvectors:
 [[-0.05874635+0.j          0.07912473+0.12660324j  0.07912473-0.12660324j
  -0.99984555+0.j        ]
 [-0.14249608+0.j          0.18335591+0.22298829j  0.18335591-0.22298829j
  -0.00250869+0.j        ]
 [-0.97591602+0.j         -0.19714467-0.03991783j -0.19714467+0.03991783j
  -0.01718134+0.j        ]
 [-0.15437502+0.j          0.92407157+0.j          0.92407157-0.j
  -0.00271783+0.j        ]]


The most ideal eigenvalue is 2.96351674e+01+0.00000000e+00j and eigenvector is [−0.05874635,−0.14249608,−0.97591602,−0.15437502] because the largest eigenvalue corresponds to the direction where there is maximized separation between classes. This is the optimal projection axis in LDA. 

Using a choice of u from 1.c, predict the class of train data and calculate the accuracy score
using the scikit-learn accuracy score function

In [5]:
from sklearn.metrics import accuracy_score

def project_data(X, u):
    return X @ u

def predict_class(X_projection, threshold=0):
    return np.where(X_projection > threshold, 1, -1)

u = eigvectors[:, 0]

X_train_projection = project_data(X_train, u)
y_train_prediction = predict_class(X_train_projection)

accuracy_train = accuracy_score(y_train, y_train_prediction)
print("Training accuracy using the first eigenvector:", accuracy_train)


Training accuracy using the first eigenvector: 0.0


Predict the class of train data using other u vectors in 1.c and calculate the accuracy. Based
on the results in 1.c and 1.d, explain if your choice of u is the correct decision

In [6]:
u = eigvectors[:, 1]

X_train_projection = project_data(X_train, u)
y_train_prediction = predict_class(X_train_projection)

accuracy_train = accuracy_score(y_train, y_train_prediction)
print("Training accuracy using the second eigenvector:", accuracy_train)


Training accuracy using the second eigenvector: 0.525


In [7]:
u = eigvectors[:, 2]

X_train_projection = project_data(X_train, u)
y_train_prediction = predict_class(X_train_projection)

accuracy_train = accuracy_score(y_train, y_train_prediction)
print("Training accuracy using the third eigenvector:", accuracy_train)


Training accuracy using the third eigenvector: 0.525


In [8]:
u = eigvectors[:, 3]

X_train_projection = project_data(X_train, u)
y_train_prediction = predict_class(X_train_projection)

accuracy_train = accuracy_score(y_train, y_train_prediction)
print("Training accuracy using the fourth eigenvector:", accuracy_train)


Training accuracy using the fourth eigenvector: 0.525


The training accuracy of the first eigenvector was the lowest at 0.0 even though it corresponded to the highest eigenvalue. The remaining eigenvectors resulted in 0.525 training accuracy which is better than the first.

Predict the test data class. Report the accuracy. Use scikit-learn LDA to classify the test
data

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=1)
lda.fit(X_train, y_train)
y_test_prediction = lda.predict(X_test)
accuracy_test = accuracy_score(y_test, y_test_prediction)
print("Test accuracy using scikit-learn LDA:", accuracy_test)


Test accuracy using scikit-learn LDA: 1.0


# Perception

In the lecture slide (54), the methods needed for the perceptron algorithm are provided (step(X) and perceptron predict(w, X)). Write a method (Perceptron fit(w, X, y, learning rate, iteration)) that fits the data and returns w

In [103]:
def step(X):
    return np.where(X >= 0, 1, -1)
def perceptron_predict(w, X):
    h = step(np.dot(X,w))
    return h

def perceptron_fit(w, X, y, learning_rate, iterations):
    for _ in range(iterations):
        for xi, target in zip(X, y):
            h = step(np.dot(w, xi))
            if h != target:
                error = target - h
                w += learning_rate * error * xi
    return w

Fit the train data and find w when the learning rate is 0.001 and iteration is 1. The learning
rate and iteration numbers can be tuned to increase the performance if necessary.

In [104]:
w = perceptron_fit(w, X_train, y_train, 0.001, 1000)
print("Weights after training:\n", w)

Weights after training:
 [0.00445822 0.00195538 0.02682563 0.00525618]


Using the final w from 2.b, generalize the model using the test set.

In [105]:
y_test_pred_perceptron = perceptron_predict(w, X_test)
accuracy_test_perceptron = accuracy_score(y_test, y_test_pred_perceptron)
print("Test accuracy using Perceptron:", accuracy_test_perceptron)

Test accuracy using Perceptron: 1.0
